In [3]:
import torch
words = open('names.txt', 'r').read().splitlines()

In [10]:
# a dictionary that maps each character to a unique integer
# print(f"{sorted(set(''.join(words)))=}")
chars = sorted(list(set(''.join(words)))) # ['a', 'b', 'c',
itos = dict(enumerate(chars, start=1))
itos[0] = '.'
stoi = {s:i for i,s in itos.items()}
# print(f"{itos=}")

In [ ]:
list(words[0])

['e', 'm', 'm', 'a']

In [ ]:
block_size = 3

X, Y = [], []
for w in words:
    context_s = block_size * '.' + w + '.'
    context_i = stoi()
    for i in range(len(w) + 1):
        ch = stoi 

In [20]:
block_size = 3
w = words[0]
itos[1]

'a'